In [ ]:
! pip install kaggle

In [1]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"reyhanfajar","key":"c907f661bc681f8466c3277d3896081a"}'}

In [10]:
! cp kaggle.json ~/.kaggle/

In [11]:
! chmod 600 ~/.kaggle/kaggle.json

In [12]:
! kaggle datasets download -d aryanrastogi7767/ecommerce-fraud-data

  0% 0.00/29.1k [00:00<?, ?B/s]
100% 29.1k/29.1k [00:00<00:00, 60.9MB/s]


In [13]:
file_name = 'ecommerce-fraud-data.zip'

# Specify the directory where you want to extract the files
extract_path = '/content/'

# Unzip the file
import zipfile
with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [14]:
import os

# Specify the directory where you extracted the files
extract_path = '/content/'

# List the contents of the directory
file_list = os.listdir(extract_path)

# Print the list of files

print("Files in the directory:")
for file in file_list:
    print(file)


Files in the directory:
.config
cust_transaction_details (1).csv
ecommerce-fraud-data.zip
kaggle.json
Customer_DF (1).csv
sample_data


In [15]:
import pandas as pd
import numpy as np


df_customer = pd.read_csv(file_list[1])
df_transact = pd.read_csv(file_list[4])
df_customer.head()

,Unnamed: 0,customerEmail,transactionId,orderId,paymentMethodId,paymentMethodRegistrationFailure,paymentMethodType,paymentMethodProvider,transactionAmount,transactionFailed,orderState
0,0,josephhoward@yahoo.com,a9lcj51r,vjbdvd,wt07xm68b,1,card,JCB 16 digit,18,0,pending
1,1,josephhoward@yahoo.com,y4wcv03i,yp6x27,wt07xm68b,1,card,JCB 16 digit,26,0,fulfilled
2,2,evansjeffery@yahoo.com,5mi94sfw,nlghpa,41ug157xz,0,bitcoin,American Express,45,0,fulfilled
3,3,evansjeffery@yahoo.com,br8ba1nu,uw0eeb,41ug157xz,0,bitcoin,American Express,23,0,fulfilled
4,4,evansjeffery@yahoo.com,a33145ss,bn44oh,y3xp697jx,1,bitcoin,VISA 16 digit,43,1,fulfilled


In [16]:
df_customer.drop(columns='Unnamed: 0',inplace=True, axis=1)
df_transact.drop(columns='Unnamed: 0',inplace=True, axis=1)

In [17]:
df_transact.head()

,customerEmail,customerPhone,customerDevice,customerIPAddress,customerBillingAddress,No_Transactions,No_Orders,No_Payments,Fraud
0,josephhoward@yahoo.com,400-108-5415,yyeiaxpltf82440jnb3v,8.129.104.40,"5493 Jones Islands\nBrownside, CA 51896",2,2,1,False
1,evansjeffery@yahoo.com,1-788-091-7546,r0jpm7xaeqqa3kr6mzum,219.173.211.202,356 Elizabeth Inlet Suite 120\nPort Joshuabury...,3,3,7,True
2,andersonwilliam@yahoo.com,024.420.0375,4m7h5ipl1shyavt6vv2r,67b7:3db8:67e0:3bea:b9d0:90c1:2b60:b9f0,"8478 Sean Ridges Apt. 441\nDavisberg, PR 72250",5,3,2,False
3,rubenjuarez@yahoo.com,670.664.8168x94985,slovx60t0i558may4ks0,95de:8565:5a66:792c:26e0:6cfb:7d87:11af,"7769 Elizabeth Bridge Apt. 343\nNortonstad, FM...",3,3,1,False
4,uchen@malone.com,1-981-877-0870,j0pd24k5h8dl2fqu0cz4,196.89.235.192,"148 Russell Lodge Apt. 445\nPort Jenniferside,...",7,7,6,True


In [18]:
new_df_customer = df_customer.drop(columns=['transactionId','orderId','paymentMethodId'], axis=1)
new_df_transact = df_transact.drop(columns=['customerPhone','customerDevice','customerIPAddress','customerBillingAddress'], axis=1)

In [19]:
list_df = [new_df_customer, new_df_transact]

for i in range(len(list_df)):
  null_count = list_df[i].isnull().sum()
  print(null_count)


customerEmail                       0
paymentMethodRegistrationFailure    0
paymentMethodType                   0
paymentMethodProvider               0
transactionAmount                   0
transactionFailed                   0
orderState                          0
dtype: int64
customerEmail      0
No_Transactions    0
No_Orders          0
No_Payments        0
Fraud              0
dtype: int64


In [20]:
final_df = new_df_customer.join(new_df_transact.set_index('customerEmail'), on='customerEmail')

In [21]:
!pip install pandas-gbq

In [22]:
from google.cloud import bigquery
from google.colab import auth

PROJECT_ID = "precise-equator-357704"  # @param {type:"string"}

auth.authenticate_user(project_id=PROJECT_ID)

In [ ]:
project_id = "dana-project-357612"
dataset_id = "fraud_data"
table_id = "fraud_dataset"

destination_table = f"{project_id}.{dataset_id}.{table_id}"
final_df.to_gbq(destination_table, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<00:00, 7781.64it/s]
